In [1]:
from urllib.request import urlopen as uReq
import requests
import lxml.html as lh
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup


In [2]:
my_url = 'https://www.imdb.com/list/ls076742571'
uClient = uReq(my_url)
pagehtml = uClient.read()
uClient.close()
s = soup(pagehtml,'html.parser')
lists_div =  s.findAll("div", {"class": "lister-list"})
print(len(lists_div)) # There should only be one div in this list of divs
films_div = lists_div[0]



1


In [3]:
# <div>...</div> is a section, more multi-line block in html
# <span ../> is a single-line block in html

# Get the list of film item detail divs, each <div ... lister-item mode-detail ... /> is a film item detail
film_item_detail_divs = films_div.findAll("div", {"class": "lister-item mode-detail"})

listoffilms = []
for film_parent_div in film_item_detail_divs:
    imdb_dict = {}
    film_content_div = film_parent_div.find("div", {"class", "lister-item-content"})
    #print(film_content_div)
    title_link = film_content_div.find("a")
    film_title = title_link.getText()
    print(film_title)
    
    rating_span = film_content_div.find("span", {"class": "ipl-rating-star__rating"})
    #print(rating_span)
    rating_str = rating_span.getText()
    #print(rating_str)
    rating = float(rating_str)
    print(rating)
    
    box_office_div = film_parent_div.find("div", {"class", "list-description"})
    #print(box_office_div)
    box_office_p = box_office_div.find("p")
    #print(box_office_p.contents)
    box_office = box_office_p.contents[0]
    print(box_office)
    
    #genre_div = film_parent_div.find("span", {"class": "genre"})
    #print(genre_div) 
    genre_div_p1 = film_parent_div.find("p")
    genres = genre_div_p1.find("span", {"class","genre"})
    genre_list = genres.getText(strip= True)
    print (genre_list)
    #genre_div_p = genre_div.getText()
    #print(genre_div_p)
    imdb_dict['title'] = film_title
    imdb_dict['rating'] = rating
    imdb_dict['boxoffice'] = box_office
    imdb_dict['genre'] = genre_list

    listoffilms.append(imdb_dict)

    #print(detail)
#print(films_div.findAll(



Titanic
7.8
$2.129 M.
Drama, Romance
Inception
8.8
$826 M.
Action, Adventure, Sci-Fi
The Revenant
8.0
$519 M.
Action, Adventure, Biography
Django Unchained
8.4
$425 M.
Drama, Western
The Wolf of Wall Street
8.2
$392 M.
Biography, Crime, Drama
Catch Me If You Can
8.1
$352 M.
Biography, Crime, Drama
The Great Gatsby
7.3
$351 M.
Drama, Romance
Once Upon a Time in Hollywood
8.0
$345 M.
Comedy, Drama
Shutter Island
8.1
$295 M.
Mystery, Thriller
The Departed
8.5
$290 M.
Crime, Drama, Thriller
The Aviator
7.5
$214 M.
Biography, Drama
Gangs of New York
7.5
$194 M.
Crime, Drama
The Man in the Iron Mask
6.5
$183 M.
Action, Adventure, Drama
Blood Diamond
8.0
$171 M.
Adventure, Drama, Thriller
Romeo + Juliet
6.7
$148 M.
Drama, Romance
The Beach
6.7
$144 M.
Adventure, Drama, Romance
Body of Lies
7.1
$115 M.
Action, Drama, Thriller
J. Edgar
6.5
$85 M.
Biography, Crime, Drama
Revolutionary Road
7.3
$75 M.
Drama, Romance
The Quick and the Dead
6.4
$19 M.
Action, Romance, Thriller
Marvin's Room
6.7
$13

In [6]:
print(len(listoffilms))

30


In [31]:
## Connect to DB server on AWS
import mysql.connector 
from mysql.connector import errorcode
import config2
## Connecting to the database
## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector.connect(
    host = config2.host,
    user = config2.user,
    passwd = config2.password
)

In [32]:
cursor = cnx.cursor()

In [67]:
## Create new DB 
db_name = 'Leonardo'
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)
        
    #above is python code that is going to connect to SQL queries w/ cursor.execute
    #cursor knows to take SQL query and send it to server to run code
    #try & except is error handling within python
    #format takes in database and tries to run it
    #if failed, except will tell you error and why it failed
    #above is a function to create a database

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name####
    else:
        print(err)
        exit(1)

In [70]:
TABLES = {}

TABLES['imdbfilms'] = \
'''CREATE TABLE IF NOT EXISTS imdbfilms (
      title varchar(200) NOT NULL PRIMARY KEY, 
      rating int,
      box_office varchar(200),
      genre varchar(200) NOT NULL
)  ENGINE=INNODB;
'''

In [71]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")


Creating table imdbfilms: OK


In [72]:
df = pd.DataFrame(listoffilms)

In [73]:
def get_review_tuples(d):
    tuples = []
    for i in range(len(d)):
        t = (d.iloc[i]['title'], d.iloc[i]['rating'], d.iloc[i]['genre'], d.iloc[i]['boxoffice'])
        tuples.append(t)
    return tuples

    

In [74]:
get_review_tuples(df)

[('Titanic', 7.8, 'Drama, Romance', '$2.129 M.'),
 ('Inception', 8.8, 'Action, Adventure, Sci-Fi', '$826 M.'),
 ('The Revenant', 8.0, 'Action, Adventure, Biography', '$519 M.'),
 ('Django Unchained', 8.4, 'Drama, Western', '$425 M.'),
 ('The Wolf of Wall Street', 8.2, 'Biography, Crime, Drama', '$392 M.'),
 ('Catch Me If You Can', 8.1, 'Biography, Crime, Drama', '$352 M.'),
 ('The Great Gatsby', 7.3, 'Drama, Romance', '$351 M.'),
 ('Once Upon a Time in Hollywood', 8.0, 'Comedy, Drama', '$345 M.'),
 ('Shutter Island', 8.1, 'Mystery, Thriller', '$295 M.'),
 ('The Departed', 8.5, 'Crime, Drama, Thriller', '$290 M.'),
 ('The Aviator', 7.5, 'Biography, Drama', '$214 M.'),
 ('Gangs of New York', 7.5, 'Crime, Drama', '$194 M.'),
 ('The Man in the Iron Mask', 6.5, 'Action, Adventure, Drama', '$183 M.'),
 ('Blood Diamond', 8.0, 'Adventure, Drama, Thriller', '$171 M.'),
 ('Romeo + Juliet', 6.7, 'Drama, Romance', '$148 M.'),
 ('The Beach', 6.7, 'Adventure, Drama, Romance', '$144 M.'),
 ('Body of 

In [75]:
cnx = mysql.connector.connect(
    host = config2.host,
    user = config2.user,
    passwd = config2.password
)

In [76]:
cursor = cnx.cursor()

In [77]:
addfilms = ("INSERT INTO Leonardo.imdbfilms" 
               "(title, rating, genre, box_office)"
               "VALUES (%s, %s, %s, %s)")
cursor.executemany(addfilms, get_review_tuples(df))
cnx.commit()